# Importation des modules

In [34]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, RepeatedKFold, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier

# Modélisation

## 1. Recherche des features importantes

In [35]:
dataset = pd.read_csv("archive/SBAnational_clean.csv")
df = dataset[['NAICS', 'NoEmp', 'NewExist', 'CreateJob', 'RetainedJob', 'FranchiseCode', 'UrbanRural', 'MIS_Status']]

/tmp/ipykernel_5337/3054430782.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv("archive/SBAnational_clean.csv")


In [36]:
df[['NewExist']] = df[['NewExist']].astype(int)
df.dtypes

/tmp/ipykernel_5337/1910921016.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['NewExist']] = df[['NewExist']].astype(int)


NAICS             int64
NoEmp             int64
NewExist          int64
CreateJob         int64
RetainedJob       int64
FranchiseCode     int64
UrbanRural        int64
MIS_Status       object
dtype: object

In [37]:
# Séparer les données en ensembles d'entraînement et de test
X = df.drop(columns=["MIS_Status"], axis=1)
y = df["MIS_Status"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify='M')

# Encodage des variables catégorielles
encoder = LabelEncoder()
for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = encoder.fit_transform(X[col])

# Entraînement du modèle
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

# Evaluation du modèle sur l'ensemble de test
accuracy = rf.score(X_test, y_test)
print("Accuracy: {:.2f}%".format(accuracy*100))

# Importance des variables
importances = rf.feature_importances_

Accuracy: 80.50%
